# Assignment 3: Hashtag based Tweet search

We will extend Assignment 2 and work on building a vector based search for hashtag based search of tweets.

Overview:
Welcome to TweetMiner, the leading organization in Twitter data analysis! As an NLP scientist in our team, you're entrusted with the task of extracting the most relevant tweets based on input hashtags. For instance, if the hashtag is "#abortion," we expect you to extract the top N (let's say N=10) tweets that truly discuss the topic of "abortion." Similarly, for a hashtag like "#politicaladvertising," your algorithm should identify and extract the top N (again, let's use N=10) tweets about "political advertising".
For this assignment your tasks are the following:

## Task 1: Use CountVectorizer (binary = true) vectorization technique and perform search

### Processing Tweets:

1. Pre-process tweets using applicable pre-processing techniques.


In [1]:
import os.path
if not os.path.isfile("preprocessed_tweets.txt"):

  # first load file! same as Assignment 2
  with open("australian_election_2019_tweets.txt") as f:
      list_tweets = f.read().splitlines()

  # pre-processing from Lab 4
  import spacy
  from nltk.corpus import stopwords
  import nltk
  nltk.download('stopwords')

  # get a list of stopwords from NLTK
  stops = set(stopwords.words('english'))

  # Load SpaCy English language model
  # this is a pipeline capable of applying morphological, lexical and syntax analysis on text

  nlp_pipeline = spacy.load("en_core_web_sm")

  def pre_process_a_single_sentence(sentence: str):
    # Lower case text
    sentence = sentence.lower()

    processed_sentence = []

    # Tokenize, and lemmatize the text
    doc = nlp_pipeline(sentence)

    for token in doc:
      # here token is an object that contains various information about each token
      # information such as lemma, pos, parse labels are available
      # we will check here if tokens are present in stopwords; if not, we will retain their lemma
      if token not in stops:
        lemmatized_token = token.lemma_
        processed_sentence.append(lemmatized_token)
    processed_sentence = " ".join (processed_sentence)
    return processed_sentence

  # remove duplicates first
  l_t = list(set(list_tweets))

  # we use regex for removing URLs, non-english text
  import re
  # credit to https://www.geeksforgeeks.org/remove-urls-from-string-in-python/
  def remove_non_english(text):
      # Define a regex pattern to find
      pattern = re.compile(r"https?://\S+|(?<=\s)[@#]|^[@#]|[^a-zA-Z0-9\s]")

      # Use the sub() method to replace
      text_without_noneg = pattern.sub("", text)

      return text_without_noneg

  ltrdru = []

  for line in l_t:
    ltrdru.append(remove_non_english(line))

  # preprocess text actual
  prepro_tweets = [pre_process_a_single_sentence(sentence) for sentence in ltrdru]


  # save lines
  with open('preprocessed_tweets.txt', 'w') as f:
      for line in prepro_tweets:
          f.write('%s\n' %line)

else:
  print("File Exists, skipping")

File Exists, skipping


I am skipping if I have the file saved because it takes 20 MINUTES to pre-process the file. But now I can simply load it to save time

In [2]:
prepro_tweets = []
with open("preprocessed_tweets.txt") as f:
    for line in f.readlines():
        # see if there is a loose blank line and skip it if so
        if (len(line.strip()) == 0):
            continue
        if line:
            prepro_tweets.append(line.strip())
print("opened")
f.close()

opened


2. Vectorize pre-processed tweets with CountVectorizer (binary = true) . This will create sparse vectors of tweets based on its vocabulary.

In [3]:
# also taken from lab 4
from sklearn.feature_extraction.text import CountVectorizer
# Define the N for N-grams
N = 1
# Initialize the CountVectorizer with N-gram range
vectorizer = CountVectorizer(ngram_range=(N, N), lowercase = False, binary = True, max_features = 5000)

# Fit and transform the corpus
vectorizer.fit(prepro_tweets)

# Check a few items in the vocabulary
vocab = vectorizer.get_feature_names_out()

# sanity check: check the list of vocabulary
print(vocab[:10])

['01' '02' '05' '07' '10' '100' '1000' '10000' '100000' '101']


In [4]:
# making a transformation of the text
tweet_vectors = []

for sentence in prepro_tweets:
  transformed_vector = vectorizer.transform([sentence])
  tweet_vectors.append(transformed_vector.toarray()[0])

print ("Transformed Corpus Samples", tweet_vectors[:5])

Transformed Corpus Samples [array([0, 0, 0, ..., 0, 0, 0]), array([0, 0, 0, ..., 0, 0, 0]), array([0, 0, 0, ..., 0, 0, 0]), array([0, 0, 0, ..., 0, 0, 0]), array([0, 0, 0, ..., 0, 0, 0])]


### Processing hashtags and conduct search:

1. Manually define a list of 10 hashtags, initiating each with the "#" symbol. Ensure the list consists of 5 single-word hashtags and 5 multiword hashtags. For multiword hashtags, capitalize the first letter of each word (e.g., #PoliticalAdvertising). 

Hashtags used: '#RenewableEnergy', '#TaxLaws', '#ParliamentaryMajority', '#coalition', '#Labor' '#Liberal', '#Auspol', '#DemocracySausage', '#ausvotes', and '#AusVotes22'

In [5]:
# list of 10 hashtags
hashtags = ['#RenewableEnergy', '#TaxLaws', '#ParliamentaryMajority', '#Coalition', '#Labor', '#Liberal', '#Auspol', '#DemocracySausage', '#Ausvotes', '#AusVotes22']

2. Remove the "#" symbol from all hashtags. If the hashtag is multiword, split it into individual words using regular expressions. Refer to the code snippet available at https://stackoverflow.com/questions/68448243/efficient-way-to-split-multi-word-hashtag-in-python

In [6]:
import re
x = 0
for tag in hashtags:
    hashtags[x] = re.sub(r'#(\w*[A-Z]\w*)', 
                         lambda m: ' '.join(re.findall('[A-Z][^A-Z]*', m.group())), tag)
    print(hashtags[x])
    x += 1

Renewable Energy
Tax Laws
Parliamentary Majority
Coalition
Labor
Liberal
Auspol
Democracy Sausage
Ausvotes
Aus Votes22


3. For each hashtag,

a. Vectorize the hashtags USING THE SAME Vectorizer that you built under "Processing Tweets". Let's call it "queryVector" 

In [7]:

query_vector = vectorizer.transform([hashtags_1]).toarray()[0]
query_vector

array([0, 0, 0, ..., 0, 0, 0])

b. Compute the pairwise similarity between the "queryVector" and  each tweet vector using inverse of Euclidean Distance (you can copy the implementation from ALTERNATIVE_Lab4 notebook). 

In [8]:
# let's define a similarity function.
# we take 1 / euclidean_distance between two vectors as the similarity

import numpy as np
def euclidean_distance_based_similarity (vector1, vector2):
    """
    Compute the Euclidean distance between two vectors.

    Parameters:
        vector1 (array-like): First vector.
        vector2 (array-like): Second vector.

    Returns:
        float: Euclidean distance between the two vectors.
    """
    return 1 / (np.linalg.norm(np.array(vector1) - np.array(vector2)))


similarity_scores = {}

for i, document_vector in enumerate(tweet_vectors):
  sim = euclidean_distance_based_similarity(document_vector, query_vector)
  similarity_scores[i] = sim

ranked_documents = sorted(similarity_scores.items(),key = lambda x: x[1] ,reverse = True)

# Let's print the top 5 documents based on ranked score

print (f"Query: {hashtags_1}")
for document_idx, score in ranked_documents[:5]:
    print (f"Document: {prepro_tweets[document_idx]}, Score: {score}")


/tmp/ipykernel_1442/912260388.py:16: RuntimeWarning: divide by zero encountered in scalar divide
  return 1 / (np.linalg.norm(np.array(vector1) - np.array(vector2)))


Query: Renewable Energy Tax Laws Parliamentary Majority Coalition Labor Liberal Auspol Democracy Sausage Ausvotes Aus Votes22
Document: yummmm, Score: inf
Document: imbeingverysarcasticrightnow, Score: inf
Document: kot5aside, Score: inf
Document: 6 405, Score: inf
Document: helenhaine, Score: inf


[(67, inf),
 (75, inf),
 (182, inf),
 (214, inf),
 (259, inf),
 (260, inf),
 (327, inf),
 (358, inf),
 (498, inf),
 (546, inf),
 (605, inf),
 (677, inf),
 (693, inf),
 (709, inf),
 (714, inf)]

c. Rank tweets based on the similarity score in ascending order. Print the top 10 most similar tweets. 

d. Repeat (a) (b) (c) for all 10 hashtags that you manually created.